# SBS SMK Test

This notebook should be used as a test for ensuring correct SBS image loading and processing before running a snakemake workflow.
Change the parameters specified at SET PARAMETERS cells and run the notebook to ensure the functions work as intended.

## Imports

In [2]:
from pathlib import Path

from ops.sbs_smk import Snake_sbs

import ops
from ops.imports_ipython import *
from ops.qc import *
import seaborn as sns
from microfilm import microplot
from microfilm.microplot import Micropanel
import matplotlib.pyplot as plt
import matplotlib

## Tile-Based SBS

### SET PARAMETERS

The following cell contains crucial variables that need to be set according to your specific experimental setup and data organization. Please review and modify these variables as needed before proceeding with the analysis.

- `SBS_CYCLES`: The number and order of SBS cycles
- `WELLS` and `TILES`: The well and tile you want to analyze (for this notebook, we're using one well/tile combination for testing)
- `CHANNELS` and `preprocess_pattern`: The channel names and file naming conventions
- `df_design`: The path to your guide RNA design file
- `DISPLAY_RANGES` and `LUTS`: The display ranges and color schemes (Look-Up Tables) for your image data

Ensure these variables accurately reflect your experimental setup to guarantee correct data processing and analysis.

In [5]:
# Define lists of cycles
SBS_CYCLES = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
CYCLE_FILES = None

# Set wells and tiles (in this notebook, one well / tile combination to test on)
WELLS = 'A1' 
TILES = 50
WILDCARDS = dict(well=WELLS, tile=TILES)

# Define channels
CHANNELS = None

# Define the file pattern
PREPROCESS_PATTERN = 'input_sbs_tif/10X_c{cycle}-SBS-{cycle}_{well}_Tile-{tile}.sbs.tif'

# Read and format guide data
DF_DESIGN_PATH = "../example_analysis/example_data/sbs/input/pool10_design.csv"
df_design = pd.read_csv(DF_DESIGN_PATH, index_col=None)
df_pool = df_design.query('dialout==[0,1]').drop_duplicates('sgRNA')
df_pool['prefix'] = df_pool.apply(lambda x: x.sgRNA[:x.prefix_length], axis=1) # 13
barcodes = df_pool['prefix']

# Define display ranges for different channels, recognized by ImageJ
DISPLAY_RANGES = [
    [500, 15000],  # Range for DAPI channel
    [100, 10000],  # Range for CY3 channel
    [100, 10000],  # Range for A594 channel
    [200, 25000],  # Range for CY5 channel
    [200, 25000]   # Range for CY7 channel
]

# Define LUTs (Lookup Tables) for different channels
LUTS = [
    ops.io.GRAY,    # Lookup table for DAPI channel
    ops.io.GREEN,   # Lookup table for CY3 channel
    ops.io.RED,     # Lookup table for A594 channel
    ops.io.MAGENTA, # Lookup table for CY5 channel
    ops.io.CYAN     # Lookup table for CY7 channel
]

In [7]:
import shutil

input_files = generate_file_paths(WELLS, TILES, SBS_CYCLES, preprocess_pattern)
all_files = [item for sublist in input_files for item in sublist]

# Print the files
for f in all_files:
    file_path = os.path.join(home, f)
    print(file_path)

move_to_path = "/lab/barcheese01/rkern/OpticalPooledScreens/example_analysis/example_data/sbs/input_tifs/"

# Print and copy the files
for f in all_files:
    file_path = os.path.join(home, f)
    print(file_path)
    
    # Construct the destination path
    dest_path = os.path.join(move_to_path, os.path.basename(file_path))
    
    # Copy the file
    shutil.copy(file_path, dest_path)

/lab/barcheese01/screens/denali/input_sbs_tif/10X_c1-SBS-1_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c2-SBS-2_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c3-SBS-3_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c4-SBS-4_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c5-SBS-5_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c6-SBS-6_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c7-SBS-7_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c8-SBS-8_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c9-SBS-9_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c10-SBS-10_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c11-SBS-11_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c1-SBS-1_A1_Tile-50.sbs.tif
/lab/barcheese01/screens/denali/input_sbs_tif/10X_c2-SBS-2_A